In [1]:
%config Completer.use_jedi = False

import sys
sys.path.insert(1,'../')
sys.path.insert(1,'../models/')


# Pytorch related
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from torch.utils import data as dt
from torchinfo import summary
import torchvision.models as pretrained_models
from alexnet_pytorch import AlexNet
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils import model_zoo



import numpy as np
import pandas as pd
import os


# Extra imports
# from pytorch_pretrained_vit import ViT
from lib.feature_extractor import FeatureExtractor
from lib.build_fe_ft_models import *
from models import barlow_twins


updated


In [2]:
!python train.py -h

updated
usage: train.py [-h] [--get-B GET_B [GET_B ...]] [--get-D GET_D [GET_D ...]]
                [--get-A GET_A [GET_A ...]] [--fine-tune N]
                [--base-model-name N] [--layer-name N] [--batch-size N]
                [--num-workers N] [--use-device N] [--total-num-epochs N]
                [--optimizer-name N] [--optimizer-lr N]
                [--optimizer-momentum N] [--optimizer-wd N] [--is-scheduler]

Evaluate model features on ImageNet

optional arguments:
  -h, --help            show this help message and exit
  --get-B GET_B [GET_B ...]
                        path element - global curvature
  --get-D GET_D [GET_D ...]
                        path element - distance between elements
  --get-A GET_A [GET_A ...]
                        path element - local curvature
  --fine-tune N         Fine Tuning or Frozen weights in base model
  --base-model-name N   base model to readout from
  --layer-name N        layer of base model to readout from
  --batch-size N       

# Hyperparameters

In [3]:
# base_model_name='alexnet-pytorch_regim_categ'

# base_model_name='alexnet-bagnet33_regim_categ'
# base_model_name='alexnet-bagnet17_regim_categ'

# base_model_name='alexnet_styim_categ'


base_model_name='alexnet-random_nodata_notask'




use_device='2'
fine_tune='true'
# optimizer_lr=0.0001

optimizer_lr=0.00001


if('vit' not in base_model_name):
    temp_input_to_base=torch.randn(2,3,512,512)
    img_dim=512
else:
    temp_input_to_base=torch.randn(2,3,384,384)
    img_dim=384
    

# Example training 1 layer for 2 epochs

In [4]:
# !python train.py --base-model-name $base_model_name --layer-name classifier.6 --use-device $use_device --total-num-epoch 10 --fine-tune $fine_tune

# Training all layers

In [5]:
def get_all_leaf_layers(model):
    
    def nested_children(m: torch.nn.Module):
        children = dict(m.named_children())
        output = {}
        if children == {}:
            # if module has no children; m is last child! :O
            return m
        else:
            # look for children from children... to the last child!
            for name, child in children.items():
                try:
                    output[name] = nested_children(child)
                except TypeError:
                    output[name] = nested_children(child)
        return output
    
    


    all_layer_name=[]
    all_layer=[]
    
    def recursive_call(all_layer_dict, add_str=''):
        for key in all_layer_dict.keys():
            if(isinstance(all_layer_dict[key],dict)):
                recursive_call(all_layer_dict[key],add_str=key+'_')
            else:
                all_layer_name.append((add_str+key))
                all_layer.append(all_layer_dict[key])
    
    
    
    all_layer_dict=nested_children(model)
    recursive_call(all_layer_dict)
    
    
    all_module_dict=dict(model.named_modules())
    
    return all_layer_name, all_layer, all_layer_dict, all_module_dict
    
    
    
    

# Building, Saving and Loading feature extractor models examples

In [6]:
base_model_name

'alexnet-random_nodata_notask'

In [7]:
base_model=get_base_model(base_model_name)

In [8]:
base_model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [9]:
leaf_modules,_,_,_=get_all_leaf_layers(base_model)
leaf_modules

['features_0',
 'features_1',
 'features_2',
 'features_3',
 'features_4',
 'features_5',
 'features_6',
 'features_7',
 'features_8',
 'features_9',
 'features_10',
 'features_11',
 'features_12',
 'avgpool',
 'classifier_0',
 'classifier_1',
 'classifier_2',
 'classifier_3',
 'classifier_4',
 'classifier_5',
 'classifier_6']

In [ ]:
for i in range(len(leaf_modules)):
# for i in range(10):
    layer_name=leaf_modules[i].replace('_','.')
    !python train.py --base-model-name $base_model_name --layer-name $layer_name --use-device $use_device --fine-tune $fine_tune --optimizer-lr $optimizer_lr

updated
  0%|          | 0/100 [00:00<?, ?it/s]
updated
  0%|          | 0/100 [00:00<?, ?it/s]
updated
  0%|          | 0/100 [00:00<?, ?it/s]
updated
  0%|          | 0/100 [00:00<?, ?it/s]
updated
  0%|          | 0/100 [00:00<?, ?it/s]
updated
  0%|          | 0/100 [00:00<?, ?it/s]
updated
  0%|          | 0/100 [00:00<?, ?it/s]
updated
  0%|          | 0/100 [00:00<?, ?it/s]


In [ ]:
print('Done')